In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat # einops 직관적으로 사용할 수 있는 차원관리 package
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

### Project input to Patches

In [5]:
# input
x = torch.randn(8, 3, 224, 224) # (batch_size, channel, height, width)
print(f'x: {x.shape}')

# b c (h s1) (w s2) # height, width를 각각 patch_size로 나누라는 말
# b (h w) (s1 s2 c) # batch_size는 그대로 두고 h*w로 펼쳐서 1차원으로 만들고(14*14=196) s1*s2*3을 펼쳐서 (16*16*3)이 된다. 
patch_size = 16
patches = rearrange(x, 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', 
                    s1=patch_size, s2=patch_size)
print(f"patches: {patches.shape}")

x: torch.Size([8, 3, 224, 224])
patches: torch.Size([8, 196, 768])


In [8]:
patch_size = 16
in_channels = 3
emb_size = 768 # channel * patch_size * patch_size

projection = nn.Sequential(
    nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size), 
    Rearrange('b e (h) (w) -> b (h w) e')
)
summary(projection, x.shape[1:], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
Total params: 590,592
Trainable params: 590,592
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 2.30
Params size (MB): 2.25
Estimated Total Size (MB): 5.12
----------------------------------------------------------------
